## Quill stuff

### Deltas

## SpaCy and TextaCy

In [28]:
doc_path = 'corpus_data/comp_ling.txt'

with open(doc_path) as f:
    input_doc = f.read()

In [29]:
import spacy
nlp = spacy.load('en')

In [198]:
import textacy

In [199]:
# input_doc = '''
# Oxygen is a chemical element with symbol O and atomic number 8. A member of the chalcogen group on the periodic table, it is a highly reactive nonmetal and oxidizing agent that forms oxides with most elements as well as other compounds.
# '''

def to_spacy_doc(raw_doc):
    '''Converts a raw string into a spaCy document'''
    return nlp(raw_doc)

def to_textacy_doc(raw_doc):
    '''Converts a raw string into a spaCy doc, then a textacy doc'''
    if isinstance(to_spacy_doc("test"), spacy.tokens.doc.Doc):
        return textacy.Doc(raw_doc)
    else:
        return textacy.Doc(nlp(raw_doc))

In [45]:
def get_named_entities(doc):
    nes = textacy.extract.named_entities(doc)
    return [ne for ne in nes]

get_named_entities(doc)

[Today, 
 
 The Association for Computational Linguistics, 
 
 ..., 1, 2, 2.1, 2.2, 2.3, 2.3.1, 2.3.2, 
 2.4, 3, 4, 5, 6, 7	References, 8	External, United States, 1950s, Russian, 1960s, one, one, one, four, one, Crucially, Pólya, modern-day, English, One, One, English, IBM, over 4.5 million words, American, English, One, two, early in the lifetime of a field of study, English, Japanese, Japanese, Japanese, October 2015, only half, Alan Turing, Turing Test, 1950, Alan Turing, one day, two, one, Turing, Today, Turing, Joseph Weizenbaum, MIT, ELIZA, One, ELIZA, Joseph Weizenbaum, MIT, 1966, Rogerian, ELIZA, ELIZA, ELIZA, first, first, Markov, translation.[25] The, German, French, first, five, ELIZA, Siri, Bayesian, Bledsoe, Browing, 1959, one, Bayesian, Mosteller, Wallace, 1963, Federalist Papers, Madison, 1971, Terry Winograd, SHRDLU, SHRDLU, NASA, LUNAR, Apollo, 1960s and 1970s, Markov, Rabiner, 1989.[32, late 70s, IBM, Bayesian, Apple, Siri, Google Translate, Social, Twitter, four, fou

In [49]:
if not isinstance(doc, textacy.doc.Doc):
        print("ok")

In [47]:
def get_readability_stats(textacy_doc):
    ts = text_stats.TextStats(textacy_doc)
    return ts.readability_stats

get_readability_stats(doc)

{'automated_readability_index': 22.352141689526015,
 'coleman_liau_index': 16.682927688311693,
 'flesch_kincaid_grade_level': 19.052077766395673,
 'flesch_readability_ease': 19.26507900874003,
 'gulpease_index': 41.068861371186955,
 'gunning_fog_index': 22.472696136278014,
 'lix': 72.24939361765064,
 'smog_index': 18.80095838887095,
 'wiener_sachtextformel': 12.265282847241465}

In [48]:
textacy.keyterms.key_terms_from_semantic_network(doc)

[('language', 0.02416255077377969),
 ('linguistic', 0.02043304879128454),
 ('computational', 0.01746222784332213),
 ('human', 0.011689631474031183),
 ('computer', 0.011480717062340147),
 ('model', 0.011188761077018177),
 ('word', 0.011081683346972649),
 ('approach', 0.008327125663422037),
 ('speech', 0.007841891597153119),
 ('program', 0.007792902818308204)]

In [52]:
def get_textacy_key_terms(doc):
    if not isinstance(doc, textacy.doc.Doc):
        doc = to_textacy_doc(doc)
    term_prob_pairs = textacy.keyterms.key_terms_from_semantic_network(doc)
    terms = [term[0] for term in term_prob_pairs]
    return terms
get_textacy_key_terms(doc)

['language',
 'linguistic',
 'computational',
 'human',
 'computer',
 'model',
 'word',
 'approach',
 'speech',
 'program']

In [ ]:
# get top 5 textacy key_terms and generate Empath category



In [252]:
import markovify
import re
from unidecode import unidecode

# corpus_path = "/corpus_data/comp_ling.txt"
# corpus_name = "comp_ling"
corpus = textacy.Corpus('en', [wiki_blob, " "])
#corpus = textacy.Corpus.add_doc(textacy.Corpus('en'), to_textacy_doc(wiki_blob))
#corpus = wiki_blob_spacy
print(corpus)

class TaggedText(markovify.Text):

    def sentence_split(self, text):
        """
        Splits full-text string into a list of sentences.
        """
        sentence_list = []
        for doc in corpus:
            sentence_list += list(doc.sents)

        return sentence_list

    def word_split(self, sentence):
        """
        Splits a sentence into a list of words.
        """
        #print(sentence)
        return ["::".join((word.orth_,word.pos_)) for word in sentence]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence

    def test_sentence_input(self, sentence):
        """
        A basic sentence filter. This one rejects sentences that contain
        the type of punctuation that would look strange on its own
        in a randomly-generated sentence. 
        """
        sentence = sentence.text
        reject_pat = re.compile(r"(^')|('$)|\s'|'\s|[\"(\(\)\[\])]")
        # Decode unicode, mainly to normalize fancy quotation marks
        if sentence.__class__.__name__ == "str":
            decoded = sentence
        else:
            decoded = unidecode(sentence)
        # Sentence shouldn't contain problematic characters
        if re.search(reject_pat, decoded): return False
        return True

    def generate_corpus(self, text):
        """
        Given a text string, returns a list of lists; that is, a list of
        "sentences," each of which is a list of words. Before splitting into 
        words, the sentences are filtered through `self.test_sentence_input`
        """
        sentences = self.sentence_split(text)
        passing = filter(self.test_sentence_input, sentences)
        runs = list(map(self.word_split, passing))
        #print(runs[:10])
        return runs

# Generated the model
model = TaggedText(corpus)
# A sentence based on the model
print(model.make_sentence())
model.make_short_sentence(max_chars=100)

Corpus(2 docs; 10783 tokens)
Most burger shops have also adopted a pizzeria - like shape , cut from whole potatoes using a specialized spiral slicer . 



'90% consumed by the early 1970s .'

In [62]:
import pandas as pd

token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in doc]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Computational,-19.579313,,,,,
1,linguistics,-19.579313,,,,,
2,is,-4.329765,Yes,,,,
3,an,-5.953294,Yes,,,,
4,interdisciplinary,-19.579313,,,,,
5,field,-9.710699,,,,,
6,concerned,-9.861534,,,,,
7,with,-5.363765,Yes,,,,
8,the,-3.425446,Yes,,,,
9,statistical,-11.639928,,,,,


In [ ]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print u'{:20} {}'.format(word, round(similarity, 3))

## Wikipedia Data

### Wikipedia wrapper (API)

In [70]:
import wikipedia
from wikipedia import DisambiguationError, PageError, RedirectError

def get_wiki_page(search_string, summary=False, content=False):
    '''Returns results from searching for search_string with wikipedia wrapper library. 
       Note: Makes a web request'''
    try:
        page = wikipedia.page(search_string)
        page_data = {"url":page.url,
                     "title":page.title}
        if content:
            page_data["content"] = page.content # Full text content of page.
        if summary:
            page_data["summary"] = page.summary # Summary section only.
    
    except DisambiguationError as e:
        return get_wiki_page(e.options[0]) #naively choose first option
    except Exception as e:
        return None
    
    return page_data


In [154]:
test_doc = "London"

In [67]:
def extract_named_entities(text):
    '''Given a text document, extracts named entities using spaCy and builds a dict of metadata for each.
    
    Example Entity Type Labels:
    ORGANIZATION	Georgia-Pacific Corp., WHO
    PERSON	Eddy Bonte, President Obama
    LOCATION	Murray River, Mount Everest
    DATE	June, 2008-06-29
    TIME	two fifty a m, 1:30 p.m.
    MONEY	175 million Canadian Dollars, GBP 10.40
    PERCENT	twenty pct, 18.75 %
    FACILITY	Washington Monument, Stonehenge
    GPE	South East Asia, Midlothian
    '''
    
    doc = nlp(text)
    named_entities = defaultdict(dict)
    for ent in doc.ents:
        ent_name = ent.text
        named_entities[ent_name]['label'] = ent.label_
        named_entities[ent_name]['text'] = ent.text
        wiki_url = None #get_wiki_page(str(ent))['url']
        if wiki_url:
            named_entities[ent_name]['url'] = wiki_url
        
    return named_entities

In [76]:
import bleach

def set_link_title(attrs, new=False):
    attrs[(None, u'title')] = u'AI-provided Link'
    return attrs

def linkify(string):
    '''Calls bleach.linkify.
    Converts urls in the input string into links. 
    Returns a string of HTML.'''
    if type(string) != str:
        raise TypeError("input should be a string")
        
    linker = Linker(callbacks=[set_link_title])
    return bleach.linkify(string)

def create_hyperlink(url, display_text, attrs=""):
    '''Optional attrs is a string of tag attributes.
       Example call:
       create_hyperlink('www.google.com', 'Google', 
       ... attrs = 'class=link_class title="Custom Title"')'''
    hyperlink_format = '<a href="{link}" {attrs}>{text}</a>'
    return hyperlink_format.format(link=url, attrs=attrs, text=display_text)

create_hyperlink('www.google.com', 'Google', attrs='class=link_class title="Custom Title"')

'<a href="www.google.com" class=link_class title="Custom Title">Google</a>'

In [80]:
### HTML-related functions
def create_hyperlink(url, display_text, attrs=""):
    '''Optional attrs is a string of tag attributes.
       
       create_hyperlink('www.google.com', 'Google', 
       ... attrs = 'class=link_class title="Custom Title"')'''
    
    hyperlink_format = '<a href="{link}" {attrs}>{text}</a>'
    return hyperlink_format.format(link=url, attrs=attrs, text=display_text)

def linkify_entity(ent_dict):
    '''Operates on extracted named entities. Returns HTML string.'''
    ent_type = ent_dict['label']
    text = ent_dict['text'] 
    url = get_wiki_page(text)['url']
    attrs = 'class="{ent_type}" title="{text}"'.format(ent_type=ent_type, text=text)
    return create_hyperlink(url, text, attrs=attrs)

In [81]:
test_doc = "London is a city in the U.K."
london = extract_named_entities(test_doc)["London"]
print(london)
print(linkify_entity(london))

{'label': 'GPE', 'text': 'London'}
<a href="https://en.wikipedia.org/wiki/London" class="GPE" title="London">London</a>


### Wikipedia XML Dump

#### http://www.heatonresearch.com/2017/03/03/python-basic-wikipedia-parsing.html

In [7]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os

PATH_WIKI_XML = '/Users/davideverling/Projects/Data'
FILENAME_WIKI = 'enwiki-latest-pages-articles.xml'
FILENAME_ARTICLES = 'articles.csv'
FILENAME_REDIRECT = 'articles_redirect.csv'
FILENAME_TEMPLATE = 'articles_template.csv'
ENCODING = "utf-8"

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

pathWikiXML = os.path.join(PATH_WIKI_XML, FILENAME_WIKI)
pathArticles = os.path.join(PATH_WIKI_XML, FILENAME_ARTICLES)
pathArticlesRedirect = os.path.join(PATH_WIKI_XML, FILENAME_REDIRECT)
pathTemplateRedirect = os.path.join(PATH_WIKI_XML, FILENAME_TEMPLATE)

totalCount = 0
articleCount = 0
redirectCount = 0
templateCount = 0
title = None
start_time = time.time()

In [ ]:
'''Begin streaming the XML file and 
write the headers for the 3 CSV files that will be built according to the data found in the XML.'''

with codecs.open(pathArticles, "w", ENCODING) as articlesFH, \
        codecs.open(pathArticlesRedirect, "w", ENCODING) as redirectFH, \
        codecs.open(pathTemplateRedirect, "w", ENCODING) as templateFH:
    articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)
    redirectWriter = csv.writer(redirectFH, quoting=csv.QUOTE_MINIMAL)
    templateWriter = csv.writer(templateFH, quoting=csv.QUOTE_MINIMAL)
    articlesWriter.writerow(['id', 'title', 'redirect'])
    redirectWriter.writerow(['id', 'title', 'redirect'])
    templateWriter.writerow(['id', 'title'])

    for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)
        if event == 'start':
            if tname == 'page':
                title = ''
                id = -1
                redirect = ''
                inrevision = False
                ns = 0
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
        else:
            if tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision:
                id = int(elem.text)
            elif tname == 'redirect':
                redirect = elem.attrib['title']
            elif tname == 'ns':
                ns = int(elem.text)
            elif tname == 'page':
                totalCount += 1
                if ns == 10:
                    templateCount += 1
                    templateWriter.writerow([id, title])
                elif len(redirect) > 0:
                    articleCount += 1
                    articlesWriter.writerow([id, title, redirect])
                else:
                    redirectCount += 1
                    redirectWriter.writerow([id, title, redirect])

        if totalCount > 1 and (totalCount % 100000) == 0:
            print("{:,}".format(totalCount))

    elem.clear()

    elapsed_time = time.time() - start_time
    print("Total pages: {:,}".format(totalCount))
    print("Template pages: {:,}".format(templateCount))
    print("Article pages: {:,}".format(articleCount))
    print("Redirect pages: {:,}".format(redirectCount))
    print("Elapsed time: {}".format(hms_string(elapsed_time)))


100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000
100,000


## Empath

In [124]:
from empath import Empath
lexicon = Empath()

In [155]:
lexicon.analyze(input_doc, normalize=True)

{'achievement': 0.0012221203788573174,
 'affection': 0.0006110601894286587,
 'aggression': 0.00030553009471432935,
 'air_travel': 0.0,
 'alcohol': 0.0,
 'ancient': 0.00030553009471432935,
 'anger': 0.0006110601894286587,
 'animal': 0.003360831041857623,
 'anonymity': 0.0006110601894286587,
 'anticipation': 0.0,
 'appearance': 0.002749770852428964,
 'art': 0.002444240757714635,
 'attractive': 0.0,
 'banking': 0.0,
 'beach': 0.0,
 'beauty': 0.00030553009471432935,
 'blue_collar_job': 0.0,
 'body': 0.0,
 'breaking': 0.00030553009471432935,
 'business': 0.010388023220287198,
 'car': 0.0006110601894286587,
 'celebration': 0.0015276504735716467,
 'cheerfulness': 0.0,
 'childish': 0.00030553009471432935,
 'children': 0.002444240757714635,
 'cleaning': 0.00030553009471432935,
 'clothing': 0.00030553009471432935,
 'cold': 0.00030553009471432935,
 'college': 0.0058050717995722576,
 'communication': 0.011610143599144515,
 'competing': 0.0006110601894286587,
 'computer': 0.00977696303085854,
 'con

In [166]:
tokens = '''artificial_intelligence machine_learning data_science'''
tokens = tokens.split(" ")
lexicon.create_category("category_name",tokens, model="reddit")

["machine_learning", "data_science", "deep_learning", "computer_vision", "Machine_learning", "natural_language_processing", "data_analysis", "Machine_Learning", "bioinformatics", "artificial_intelligence", "neural_nets", "big_data", "neural_networks", "Computer_science", "AI_research", "software_engineering", "robotics", "programming", "computer_science", "complexity_theory", "computing", "genetic_algorithms", "quantum_computing", "computer_scientists", "computational", "computer_programming", "software_development", "computer_architecture", "cognitive_science", "data_scientist", "control_theory", "data_analytics", "distributed_systems", "practical_applications", "econometrics", "applied_mathematics", "scientific_computing", "information_technology", "algorithms", "subfield", "theoretical_physics", "growing_field", "other_fields", "mathematics", "computer_scientist", "synthetic_biology", "distributed_computing", "cryptography", "neuroscience", "Artificial_Intelligence", "numerical_anal

In [184]:
import io
from contextlib import redirect_stdout

def capture_from_stdout(function):
    f = io.StringIO()
    with redirect_stdout(f):
        function()
    return f.getvalue()

def category_from_keywords(com_sep_keyword_string, model='all'):
    '''Call Empath's create category. Model options: "fiction","nytimes","reddit","all"   '''
    keywords = com_sep_keyword_string.split(",") #split into individual items with comma separator
    
    #replace spaces with underscores for Empath's lexicon format
    keywords = [keyword.replace(" ", "_") for keyword in keywords]

    category_name = keywords[0] + " " + keywords[1] #name the category after the first two keywords
    if model  == 'all':
        category_terms = []
        for model in ['reddit','nytimes','fiction']:
            f = io.StringIO()
            with redirect_stdout(f):
                lexicon.create_category(category_name, keywords, model=model, write=False)
            model_terms = f.getvalue().strip().replace("[]","")
            if model_terms:
                model_terms = model_terms[1:-1] #exclude enclosing brackets
                model_terms = model_terms.replace('"','').split(", ")
                category_terms.append(model_terms)
    else:
        f = io.StringIO()
        with redirect_stdout(f):
            lexicon.create_category(category_name, keywords, model=model)
        category_terms = f.getvalue().strip()
    
    category_terms = [term for model_terms in category_terms for term in model_terms] #flatten lists
    category_terms = [term.replace('_',' ') for term in category_terms] #re-separate on underscores
    return category_terms

keywords = "hamburgers,french fries,soda"
expanded_keywords = category_from_keywords(keywords)
print(keywords)
print(expanded_keywords)

hamburgers,french fries,soda
['french fries', 'French fries', 'potato chips', 'fries', 'hamburgers', 'hotdogs', 'hot dogs', 'ketchup', 'nacho cheese', 'ranch dressing', 'chicken nuggets', 'sandwiches', 'nachos', 'bagels', 'pop tarts', 'oreos', 'white bread', 'chicken wings', 'condiments', 'crackers', 'tater tots', 'hummus', 'onion rings', 'tortilla chips', 'soda', 'tomato soup', 'corn dogs', 'sandwich', 'bread sticks', 'pickles', 'bacon', 'milkshakes', 'ice cream', 'pretzels', 'beef jerky', 'cereal', 'garlic bread', 'ramen', 'salad dressing', 'hamburger', 'frozen pizza', 'chicken strips', 'Oreos', 'guacamole', 'pancakes', 'icecream', 'salad', 'soup', 'buffalo wings', 'cheeseburger', 'chicken fingers', 'nutella', 'sandwhich', 'hot wings', 'saltines', 'instant noodles', 'cheeseburgers', 'cheese fries', 'soda', 'lunch meat', 'peanut butter', 'deli meats', 'bread', 'PB&amp;J', 'apple sauce', 'chicken sandwich', 'pepperoni pizza', 'crisps', 'hot sauce', 'apple slices', 'Fritos', 'pizza', 'm

In [194]:
def get_wiki_text_for_keywords(keywords):
    all_text = ""
    if type(keywords)==str:
        keywords = keywords.split(",")
        
    print(keywords)    
    for keyword in keywords:
        wiki_data = get_wiki_page(keyword, summary=True, content=True)
        if wiki_data != None:
            try:
                #summary = wiki_data['summary']
                content = wiki_data['content']
                all_text += content
            except KeyError:
                continue
    
    return all_text

wiki_blob = get_wiki_text_for_keywords(keywords)
        

['hamburgers', 'french fries', 'soda']


/Users/davideverling/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: no parser specified warning
  markup_type=markup_type))


## Markovify

In [197]:
# generate a corpus for markovify by reading wikipedia articles for lexical category words
import markovify
text = wiki_blob

# Build the model.
text_model = markovify.Text(text)

# Print five randomly-generated sentences
for i in range(5):
    print(text_model.make_sentence())

The heat will not be as an alternative.
The hamburger, usually fried, is served between two slices of bread on the dinner menu.
Lotteria is a local version of fish and chips.
This is where the burger happened largely at the St. Louis World's Fair.
Poutine has a growing number of different cheeses, toppings, and sauces.


## Synonym Generation

### Wordnet Synsets

In [185]:
from nltk.corpus import wordnet as wn

for i,j in enumerate(wn.synsets('java')):
    print("Meaning",i, "NLTK ID:", j.name())
    print("Definition:",j.definition())
    print("Synonyms:", ", ".join(j.lemma_names()))

Meaning 0 NLTK ID: java.n.01
Definition: an island in Indonesia to the south of Borneo; one of the world's most densely populated regions
Synonyms: Java
Meaning 1 NLTK ID: coffee.n.01
Definition: a beverage consisting of an infusion of ground coffee beans
Synonyms: coffee, java
Meaning 2 NLTK ID: java.n.03
Definition: a platform-independent object-oriented programming language
Synonyms: Java


In [255]:
for ss in wn.synsets('small'):
    print(ss, ss.examples())
    for sim in ss.similar_tos():
        print('    {}'.format(sim))
        


Synset('small.n.01') []
Synset('small.n.02') []
Synset('small.a.01') ['a little dining room', 'a little house', 'a small car', 'a little (or small) group']
    Synset('atomic.s.03')
    Synset('bantam.s.01')
    Synset('bitty.s.01')
    Synset('dinky.s.01')
    Synset('dwarfish.s.01')
    Synset('elfin.s.02')
    Synset('gnomish.s.01')
    Synset('half-size.s.01')
    Synset('infinitesimal.s.01')
    Synset('lesser.s.02')
    Synset('micro.s.01')
    Synset('microscopic.s.04')
    Synset('miniature.s.01')
    Synset('minuscule.s.03')
    Synset('olive-sized.s.01')
    Synset('pocket-size.s.02')
    Synset('puny.s.02')
    Synset('slender.s.04')
    Synset('small-scale.s.01')
    Synset('smaller.s.01')
    Synset('smallish.s.01')
    Synset('subatomic.s.02')
    Synset('undersize.s.01')
Synset('minor.s.10') ['a small business', 'a newspaper with a modest circulation', 'small-scale plans', 'a pocket-size country']
    Synset('limited.a.01')
Synset('little.s.03') ['what a big little boy y

In [268]:
from itertools import chain

for i,j in enumerate(wn.synsets('language')):
    print("Meaning",i, "NLTK ID:", j.name())
    hypernyms = list(chain(*[l.lemma_names() for l in j.hypernyms()]))
    hyponyms = list(chain(*[l.lemma_names() for l in j.hyponyms()]))
    print("Hypernyms:", ", ".join(hypernyms))
    print("Hyponyms:", ", ".join(hyponyms))

Meaning 0 NLTK ID: language.n.01
Hypernyms: communication
Hyponyms: artificial_language, barrage, bombardment, outpouring, onslaught, dead_language, indigenous_language, lingua_franca, interlanguage, koine, metalanguage, native_language, natural_language, tongue, object_language, target_language, sign_language, signing, slanguage, source_language, string_of_words, word_string, linguistic_string, superstrate, superstratum, usage, words
Meaning 1 NLTK ID: speech.n.02
Hypernyms: auditory_communication
Hyponyms: conversation, dictation, discussion, give-and-take, word, idiolect, monologue, non-standard_speech, pronunciation, orthoepy, saying, expression, locution, soliloquy, monologue, spell, magic_spell, magical_spell, charm, words
Meaning 2 NLTK ID: lyric.n.01
Hypernyms: text, textual_matter
Hyponyms: love_lyric
Meaning 3 NLTK ID: linguistic_process.n.02
Hypernyms: higher_cognitive_process
Hyponyms: reading
Meaning 4 NLTK ID: language.n.05
Hypernyms: faculty, mental_faculty, module
Hypon

### PyDictionary

In [58]:
from PyDictionary import PyDictionary
from collections import defaultdict
from pprint import pprint

def dict_from_doc_tokens(unigram_tokens):
    '''Calls PyDictionary (which calls thesaurus.com) to retrieve synonyms'''
    pydict=PyDictionary(unigram_tokens)
    
    dictionary = defaultdict(dict)
    for word in unigram_tokens:
        meaning = pydict.meaning(word)
        synonyms = pydict.synonym(word)
        dictionary[word] = {"meaning":meaning, "synonyms":synonyms}
    return dictionary

def get_meanings(word, pos="all"):
    '''Returns meaning definitions from an existing pydictionary, or None if no meanings found'''
    meanings = dictionary[word]['meaning']

    if pos.startswith("all"):
        return meanings
    if pos.startswith("N"):
        return meanings['Noun']
    if pos.startswith("V"):
        return meanings["Verb"]
    if pos.startswith("J"):
        return meanings["Adjective"]
    if pos.startswith("RB"):
        return meanings["Adverb"]
    
    return meanings

tokens = ['alabaster']
dictionary = dict_from_doc_tokens(tokens)
pprint(get_meanings(tokens[0]))


/Users/davideverling/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: no parser specified warning
  markup_type=markup_type))


{'Adjective': ['of or resembling alabaster'],
 'Noun': ['a compact fine-textured, usually white gypsum used for carving',
          'a hard compact kind of calcite',
          'a very light white']}


## Proselint

In [243]:
import proselint
from pprint import pprint

suggestions = proselint.tools.lint(input_doc)
pprint(suggestions)

[('typography.symbols.ellipsis',
  "'...' is an approximation, use the ellipsis symbol '…'.",
  8,
  1,
  1255,
  1257,
  2,
  'warning',
  None),
 ('typography.symbols.curly_quotes',
  'Use curly quotes “”, not straight quotes "". Found 4 times elsewhere.',
  70,
  899,
  14585,
  14587,
  2,
  'warning',
  '“ or ”'),
 ('typography.symbols.curly_quotes',
  'Use curly quotes “”, not straight quotes "".',
  87,
  621,
  19061,
  19063,
  2,
  'warning',
  '“ or ”')]


## TextGenRNN

In [ ]:
from textgenrnn import textgenrnn

In [220]:
%%time
textgen = textgenrnn()

CPU times: user 779 ms, sys: 45.3 ms, total: 825 ms
Wall time: 845 ms


In [222]:
%%time
textgen.generate(5)

The street to the country who was a company with the successful to the state and the president and the star the support to the street and so many company and the first time to be a star to the stran

I want to be a lot of the most of the first time to start the most the star for the stream in the street of the sunders and be a beautiful things and the first time to start the states with a presid

I was been a start of the company is a stream and a lot in the first time to be a comparing the statement of the control of the state state of the store in the most favorite and a guy and the statem

The president and the compare the story of the Season 2 on THE REALLY OUT on Democratic Contrest Card in the president and the star to the stream to the star to the street to show the best of the be

A comparison of the country of the story of the story of the second and he was the stream in the street to make the world was a starts and being a months at the state and show and a stranger in the 



In [224]:
generated_texts = textgen.generate(n=5, prefix="Machine learning", temperature=0.5, return_as_list=True)
pprint(generated_texts)

['Machine learning decline of a moment when you did the most little in the '
 'grick down.',
 'Machine learning the an about the police in the one of the most meeting for '
 'the look of the deal to make the first company of changed by the behind 10 '
 'thank you say they should be about to look at t',
 'Machine learning a with the opener and having the country means the standow '
 'in a family of the morning Space Are Official Weekend',
 'Machine learning a shows at the wedding of its new first time to ever are '
 'the states to the pressure course and believe the moment and display because '
 'of the first time the street how to be comments',
 'Machine learning a tonight than an a world and share to clea...']


In [225]:
texts = [input_doc]

textgen.train_on_texts(texts, verbose=0)

In [251]:
print("---\nNormal Output\n---")
textgen.generate(5)
print("---\nHigh Temperature Output\n---")
textgen.generate(5, temperature=1.0)
print("---\nPrefix Output\n---")
textgen.generate(5, prefix="N")

---
Normal Output
---
Computational linguistics is often to develop language the probabilities for the understanding of the probability that it is also used to create a system's linguistic problem and computer program of

Computational linguistics is often to also been developed by computers to understanding human language was called a project conversational approach.

The ability to the structure of these scientist 

Computational linguistics is also been developed in the probabilities which also many has been acoustically been consisting an effort to scope of a computational and so the understanding of the user

Computational linguistics is often to develop language which computational linguistics is only also used to create as the children and computational linguistics as some of the problem of language wh

Computational linguistics is often to create a system's linguistic development of a computational and computer probabilities that can make a computational approach to this enviro

## Extractive Summarization

In [92]:
from gensim.summarization import summarize, keywords

In [98]:
def extract_summary(text, ratio=0.25):
    '''Wraps gensim summarize()'''
    return summarize(text, ratio)
extract_summary(input_doc)

"Computational linguistics originated with efforts in the United States in the 1950s to use computers to automatically translate texts from foreign languages, particularly Russian scientific journals, into English.[3] Since computers can make arithmetic calculations much faster and more accurately than humans, it was thought to be only a short matter of time before they could also begin to process language.[4] Computational and quantitative methods are also used historically in attempted reconstruction of earlier forms of modern languages and subgrouping modern languages into language families.\nUsing models, it has been shown that languages can be learned with a combination of simple input presented incrementally as the child develops better memory and longer attention span.[16] This was simultaneously posed as a reason for the long developmental period of human children.[16] Both conclusions were drawn because of the strength of the neural network which the project created.\nCruciall

In [65]:
keywords(input_doc).split()

['computers',
 'computational',
 'linguistics',
 'linguistic',
 'linguists',
 'natural',
 'language',
 'models',
 'model',
 'modeled',
 'languages',
 'particularly',
 'word',
 'words',
 'translations',
 'translated',
 'approaches',
 'human',
 'humans',
 'structural',
 'structure',
 'structures',
 'speech',
 'programs',
 'program',
 'programming',
 'based',
 'base',
 'rule',
 'modeling',
 'translate',
 'texts',
 'processing',
 'process',
 'include',
 'including',
 'machine',
 'translation',
 'development',
 'developing',
 'develops',
 'develop',
 'developed',
 'developer',
 'intelligence',
 'intelligently',
 'fields',
 'information',
 'theoretical',
 'text',
 'interactive',
 'approach',
 'examples',
 'example',
 'study',
 'studies',
 'studied',
 'testing',
 'test',
 'converse',
 'naturally',
 'origins',
 'originated',
 'originally',
 'original',
 'learned',
 'learn',
 'learning',
 'attempted',
 'attempts',
 'attempting',
 'attempt',
 'search',
 'searches',
 'work',
 'works',
 'included'